In [1]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

import time
from tqdm import tqdm_notebook

from bs4 import BeautifulSoup

In [2]:
# 홈페이지를 selenium으로 열기
driver = webdriver.Chrome()
driver.get('https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix/TV-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8?genres=drm&exclude_genres=ani,doc,rly&release_year_until=2023')


# 팝업창 닫기
try:
    driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
except NoSuchElementException:
    pass

time.sleep(1)

# 현재 페이지 높이 가져오기
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 새로운 콘텐츠가 로드되기를 기다림
    time.sleep(2)
    
    # 새로운 페이지 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 새로운 높이가 이전 높이와 같으면, 더 이상 로드할 콘텐츠가 없다는 의미
    if new_height == last_height:
        break
    
    last_height = new_height

# 현재 페이지 내의 정보를 저장
raw = driver.page_source

# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

driver.close()

# url_list 만들기
base_url = 'https://www.justwatch.com'
url_list = []

# 타이틀 리스트 내의 URL을 추출
for i in tqdm_notebook(range(2, len(soup.select('.title-list-grid')[0]) + 1)):
    tmp = soup.select_one('#base > div.title-list.title-list--CLS-block > div:nth-child(2) > div > div > div:nth-child(1) > div > div:nth-child(' + str(i) + ') > a')
    if tmp:
        tmp_url = tmp.get('href')
        if tmp_url:
            url = base_url + tmp_url
            url_list.append(url)

C:\Users\rbfl7\AppData\Local\Temp\ipykernel_21540\4057033386.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(2, len(soup.select('.title-list-grid')[0]) + 1)):


  0%|          | 0/1090 [00:00<?, ?it/s]

In [3]:
# 개수 확인
len(url_list)

1090

In [4]:
# 각 영화 데이터
movie_list = []

for x in tqdm_notebook(url_list):
    driver = webdriver.Chrome()
    driver.get(x)
    
    # 팝업창 닫기
    try:
        driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
    except NoSuchElementException:
        pass

    time.sleep(1.5)

    raw = driver.page_source
    soup = BeautifulSoup(raw, 'html.parser')

    # 'detail-infos' 클래스의 모든 'div' 태그를 찾습니다.
    detail_infos = soup.find_all('div', class_='detail-infos')

    # 중복을 제거하기 위해 집합(set)을 사용합니다.
    unique_detail_infos = list({str(info): info for info in detail_infos}.values())

    # 정보를 저장할 변수 초기화
    play_time_div = None
    genre_div = None
    rating_div = None
    country_div = None

    # 필요한 정보를 가진 'div' 태그를 각각 찾습니다.
    for info in unique_detail_infos:
        heading = info.find('h3', class_='detail-infos__subheading')
        if heading:
            if '시간' in heading.text:
                play_time_div = info.find('div', class_='detail-infos__value')
            elif '장르' in heading.text:
                genre_div = info.find('div', class_='detail-infos__value')
            elif '연령' in heading.text:
                rating_div = info.find('div', class_='detail-infos__value')
            elif 'Production' in heading.text:
                country_div = info.find('div', class_='detail-infos__value')

    # 정보를 텍스트로 가져옵니다.
    play_time = play_time_div.text.strip() if play_time_div else None
    genre = genre_div.text.strip() if genre_div and genre_div.text.strip() else None
    rating = rating_div.text.strip() if rating_div else None
    country = country_div.text.strip() if country_div else None

    # 추가 정보 추출
    title_block = soup.find('div', class_='title-block')
    korean_title_element = title_block.find('h1') if title_block and title_block.find('h1') else None
    year_element = title_block.find('span',class_='text-muted') if title_block and title_block.find('span') else None
    original_title_element = title_block.find('h3') if title_block and title_block.find('h3') else None
    seasons_element = soup.find('h2', class_='subheading')

    korean_title = korean_title_element.text.strip() if korean_title_element else None
    year = year_element.text[2:-2].strip() if year_element else None
    original_title = original_title_element.text[5:].strip() if original_title_element else None
    seasons = seasons_element.text[:-5].strip() if seasons_element else None

    # 영화 데이터 저장
    movie_data = {
        "title": korean_title,
        "original_title": original_title,
        "year": year,
        "season_episode": seasons,
        "runtime": play_time,
        "genre": genre,
        "age_rating": rating,
        "Production country": country
    }
    
    movie_list.append(movie_data)

# DataFrame 생성
jw_df = pd.DataFrame(movie_list)

# 드라이버 종료
driver.quit()

C:\Users\rbfl7\AppData\Local\Temp\ipykernel_21540\312218843.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm_notebook(url_list):


  0%|          | 0/1090 [00:00<?, ?it/s]

In [5]:
jw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               980 non-null    object
 1   original_title      767 non-null    object
 2   year                980 non-null    object
 3   season_episode      980 non-null    object
 4   runtime             980 non-null    object
 5   genre               980 non-null    object
 6   age_rating          729 non-null    object
 7   Production country  980 non-null    object
dtypes: object(8)
memory usage: 68.2+ KB


In [16]:
# 저장
jw_df.to_csv('../data/justwatch_1.csv', sep="/", encoding="utf-8")

jw_df.to_excel('../data/justwatch_1.xlsx')

## 하위 코드는 크롤링 중 놓친 데이터 다시 수집

In [12]:
miss_url_list = [url_list[i] for i in range(len(jw_df)) if jw_df.iloc[i]["title"] is None]
len(miss_url_list)

110

In [14]:
# 각 빠진 영화 데이터
miss_movie_list = []

for x in tqdm_notebook(miss_url_list):
    driver = webdriver.Chrome()
    driver.get(x)
    
    # 팝업창 닫기
    try:
        driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
    except NoSuchElementException:
        pass

    time.sleep(1.5)

    raw = driver.page_source
    soup = BeautifulSoup(raw, 'html.parser')

    # 'detail-infos' 클래스의 모든 'div' 태그를 찾습니다.
    detail_infos = soup.find_all('div', class_='detail-infos')

    # 중복을 제거하기 위해 집합(set)을 사용합니다.
    unique_detail_infos = list({str(info): info for info in detail_infos}.values())

    # 정보를 저장할 변수 초기화
    play_time_div = None
    genre_div = None
    rating_div = None
    country_div = None

    # 필요한 정보를 가진 'div' 태그를 각각 찾습니다.
    for info in unique_detail_infos:
        heading = info.find('h3', class_='detail-infos__subheading')
        if heading:
            if '시간' in heading.text:
                play_time_div = info.find('div', class_='detail-infos__value')
            elif '장르' in heading.text:
                genre_div = info.find('div', class_='detail-infos__value')
            elif '연령' in heading.text:
                rating_div = info.find('div', class_='detail-infos__value')
            elif 'Production' in heading.text:
                country_div = info.find('div', class_='detail-infos__value')

    # 정보를 텍스트로 가져옵니다.
    play_time = play_time_div.text.strip() if play_time_div else None
    genre = genre_div.text.strip() if genre_div and genre_div.text.strip() else None
    rating = rating_div.text.strip() if rating_div else None
    country = country_div.text.strip() if country_div else None

    # 추가 정보 추출
    title_block = soup.find('div', class_='title-block')
    korean_title_element = title_block.find('h1') if title_block and title_block.find('h1') else None
    year_element = title_block.find('span',class_='text-muted') if title_block and title_block.find('span') else None
    original_title_element = title_block.find('h3') if title_block and title_block.find('h3') else None
    seasons_element = soup.find('h2', class_='subheading')

    korean_title = korean_title_element.text.strip() if korean_title_element else None
    year = year_element.text[2:-2].strip() if year_element else None
    original_title = original_title_element.text[5:].strip() if original_title_element else None
    seasons = seasons_element.text[:-5].strip() if seasons_element else None

    # 영화 데이터 저장
    movie_data = {
        "title": korean_title,
        "original_title": original_title,
        "year": year,
        "season_episode": seasons,
        "runtime": play_time,
        "genre": genre,
        "age_rating": rating,
        "Production country": country
    }
    
    miss_movie_list.append(movie_data)

# DataFrame 생성
miss_df = pd.DataFrame(miss_movie_list)

# 드라이버 종료
driver.quit()

C:\Users\rbfl7\AppData\Local\Temp\ipykernel_21540\1829438500.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm_notebook(miss_url_list):


  0%|          | 0/110 [00:00<?, ?it/s]

In [15]:
miss_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               110 non-null    object
 1   original_title      88 non-null     object
 2   year                110 non-null    object
 3   season_episode      110 non-null    object
 4   runtime             110 non-null    object
 5   genre               110 non-null    object
 6   age_rating          74 non-null     object
 7   Production country  109 non-null    object
dtypes: object(8)
memory usage: 7.0+ KB


In [18]:
# 저장
miss_df.to_csv('../data/justwatch_2.csv', sep="/", encoding="utf-8")

miss_df.to_excel('../data/justwatch_2.xlsx')

In [28]:
# 병합
justWatch = jw_df[jw_df['title'].notnull()]
justWatch = pd.concat([justWatch, miss_df])
justWatch.reset_index(drop=True,inplace=True)

In [29]:
justWatch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               1090 non-null   object
 1   original_title      855 non-null    object
 2   year                1090 non-null   object
 3   season_episode      1090 non-null   object
 4   runtime             1090 non-null   object
 5   genre               1090 non-null   object
 6   age_rating          803 non-null    object
 7   Production country  1089 non-null   object
dtypes: object(8)
memory usage: 68.2+ KB


In [30]:
# 저장
justWatch.to_csv('../data/justwatch.csv', sep="/", encoding="utf-8")

justWatch.to_excel('../data/justwatch.xlsx')

In [32]:
just = pd.read_csv('../data/justwatch.csv', sep="/", encoding='utf-8', index_col=0)
just.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국
